In [1]:
from model_converter import convert_model_to_int4

convert_model_to_int4("./models/transformers/THUDM__glm-4-9b-chat-hf", output_dir="./models/transformers/genai-archive__glm-4-9b-chat-hf-4bit")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

모델이 성공적으로 8비트로 변환되어 './models/transformers/genai-archive__glm-4-9b-chat-hf-4bit'에 저장되었습니다.


True

In [2]:
from huggingface_hub import create_repo, upload_folder

create_repo(repo_id="bean980310/glm-4-9b-chat-hf-int4")

RepoUrl('https://huggingface.co/bean980310/glm-4-9b-chat-hf-int4', endpoint='https://huggingface.co', repo_type='model', repo_id='bean980310/glm-4-9b-chat-hf-int4')

In [ ]:
from optimum.quanto import freeze, qint8, quantize
from diffusers import StableDiffusionXLPipeline
from safetensors import serialize_file
import torch

pipeline = StableDiffusionXLPipeline.from_pretrained(
    "cagliostrolab/animagine-xl-3.1", 
    torch_dtype=torch.bfloat16,
).to("cpu")
quantize(pipeline.unet, weights=qint8)
freeze(pipeline.unet)
quantize(pipeline.text_encoder, weights=qint8)
freeze(pipeline.text_encoder)
quantize(pipeline.text_encoder_2, weights=qint8)
freeze(pipeline.text_encoder_2)


In [25]:
pipeline.save_pretrained("diffusion_models/animagine-xl-3.1-int8", safe_serialization=True)

In [1]:
from optimum.quanto import QuantizedDiffusersModel
from diffusers import StableDiffusionXLPipeline
from safetensors import serialize_file
import torch

class QuantizedStableDiffusionXLPipeline(QuantizedDiffusersModel):
    base_class = StableDiffusionXLPipeline

In [ ]:
# safetensors 파일에서 파이프라인 로드
pipeline_loaded = QuantizedStableDiffusionXLPipeline.from_pretrained(
    "diffusion_models/animagine-xl-3.1-int8",
    torch_dtype=torch.float16,
).to("mps")  # 또는 "cuda"

In [ ]:
from safetensors.torch import load_file
unet_safetensors_path='diffusion_models/animagine-xl-3.1-int8/unet/diffusion_pytorch_model.safetensors'

unet_state_dict_loaded = load_file(unet_safetensors_path)
pipeline_loaded.unet.load_state_dict(unet_state_dict_loaded)

In [9]:

prefixed_unet_state_dict = {f"unet.{k}": v.cpu() for k, v in unet_state_dict.items()}
prefixed_text_encoder_state_dict = {f"text_encoder.{k}": v.cpu() for k, v in text_encoder_state_dict.items()}
prefixed_text_encoder_2_state_dict = {f"text_encoder_2.{k}": v.cpu() for k, v in text_encoder_state_dict.items()}

# 전체 state_dict 결합
combined_state_dict = {**prefixed_unet_state_dict, **prefixed_text_encoder_state_dict}

combined_state_dict_2 = {**combined_state_dict, **prefixed_text_encoder_2_state_dict}

In [ ]:
from safetensors.torch import save_file

# safetensors 파일로 저장
safetensors_path = "diffusion_models/animagine-xl-3.1-int8.safetensors"
save_file(combined_state_dict_2, safetensors_path)